In [44]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import datetime as dt
import json

In [29]:
# Step 2: Retrieve the token
token = os.environ["openai_key"]

In [31]:
client = OpenAI(api_key=token)  # will now pick up OPENAI_API_KEY from env


In [ ]:
available = [m.id for m in client.models.list().data]
print(sorted(available)[:50])  # sample

['babbage-002', 'dall-e-2', 'dall-e-3', 'davinci-002', 'gpt-3.5-turbo', 'gpt-3.5-turbo-0125', 'gpt-3.5-turbo-1106', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo-instruct-0914', 'gpt-4.1', 'gpt-4.1-2025-04-14', 'gpt-4.1-mini', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-nano', 'gpt-4.1-nano-2025-04-14', 'gpt-4o', 'gpt-4o-2024-05-13', 'gpt-4o-2024-08-06', 'gpt-4o-2024-11-20', 'gpt-4o-audio-preview', 'gpt-4o-audio-preview-2024-10-01', 'gpt-4o-audio-preview-2024-12-17', 'gpt-4o-audio-preview-2025-06-03', 'gpt-4o-mini', 'gpt-4o-mini-2024-07-18', 'gpt-4o-mini-audio-preview', 'gpt-4o-mini-audio-preview-2024-12-17', 'gpt-4o-mini-search-preview', 'gpt-4o-mini-search-preview-2025-03-11', 'gpt-4o-mini-transcribe', 'gpt-4o-mini-tts', 'gpt-4o-search-preview', 'gpt-4o-search-preview-2025-03-11', 'gpt-4o-transcribe', 'gpt-5', 'gpt-5-2025-08-07', 'gpt-5-chat-latest', 'gpt-5-mini', 'gpt-5-mini-2025-08-07', 'gpt-5-nano', 'gpt-5-nano-2025-08-07', 'gpt-audio', 'gpt-audio-2025-08-28', 'gpt-image-1

In [35]:
MODEL_PARSE = "gpt-4o-mini"  # Stage 1
MODEL_PLAN  = "o3"           # Stage 2

# for your existing generate_events():
MODEL = MODEL_PARSE
TEMPERATURE = 0


In [ ]:
TEMPERATURE = 0.3
MAX_TOKENS = 500
topic = ""

In [36]:
# prompt engineering
# prompt system message
SYSTEM_PROMPT = """You are a calendar event parser that outputs ONLY a JSON object with one key: "events". Never include prose, comments, or markdown."""
# main prompt 
USER_TEMPLATE = """
Return ONLY a JSON object with a single key "events": an array of event objects. If none, return {"events": []}.

EVENT SCHEMA (each object)
{
  "title": "string",
  "event_date": "YYYY-MM-DD",
  "event_time": "HH:MM" | "",
  "end_time": "HH:MM" | "",
  "end_date": "YYYY-MM-DD",
  "timezone": "",                     // leave "" unless explicitly given
  "location": "string",
  "invitees": ["email", ...],
  "notifications": [{"method":"email|popup","minutes":int}],
  "recurrence": "RRULE string or ''",
  "confidence": 0.0-1.0
}

RULES
- Times: 24-hour; "noon"→"12:00"; "midnight"→"00:00".
- Anchor relative dates to TODAY={TODAY_ISO}.
- Do NOT invent timezones; use "" unless explicitly given.
- For time ranges ("7-9pm"), set event_time to start ("19:00") and leave end_time "".
- RRULE examples:
  - Weekly Monday: "RRULE:FREQ=WEEKLY;BYDAY=MO"
  - Weekdays: "RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR"
  - Daily x10: "RRULE:FREQ=DAILY;COUNT=10"
  - 1st Friday monthly: "RRULE:FREQ=MONTHLY;BYDAY=1FR"
  - None: ""

CONFIDENCE
Scale 0-1: high when explicit/consistent; medium when minor inference; low when ambiguous.

OUTPUT SHAPE (strict)
{"events":[ /* zero or more event objects per schema */ ]}
 
INPUT
{USER_TEXT}

""".strip()

In [23]:
def build_user_prompt(user_text: str, today_iso: str) -> str:
    return USER_TEMPLATE.replace("{TODAY_ISO}", today_iso).replace("{USER_TEXT}", user_text)

In [24]:
def safe_json_parse(raw: str):
    s = raw.strip()
    if s.startswith("```"):
        # strip code fences defensively (rare with JSON mode, but cheap)
        s = s.strip("`")
        s = s[s.find("\n")+1:]
        if s.endswith("```"):
            s = s[:s.rfind("```")]
    try:
        obj = json.loads(s)
        # Expect {"events": [...]}
        if isinstance(obj, dict) and "events" in obj and isinstance(obj["events"], list):
            return obj["events"]
        # Fallback: if provider returns a top-level list, accept it
        if isinstance(obj, list):
            return obj
        return []
    except Exception:
        return []

In [41]:
USE_MOCK = True  # flip to False when you have credits

In [47]:

def mock_generate_events(user_text: str, today_iso: str):
    # Minimal, deterministic mock for smoke tests
    if "Dinner with John" in user_text:
        return [{
            "title": "Dinner with John",
            "event_date": today_iso,
            "event_time": "19:00",
            "end_time": "",
            "end_date": today_iso,
            "timezone": "",
            "location": "Boston Pizza",
            "invitees": ["alex@example.com"],
            "notifications": [{"method": "popup", "minutes": 10}],
            "recurrence": "",
            "confidence": 0.85
        }]
    return []

def generate_events(user_text: str, today: dt.date | None = None):
    today = today or dt.date.today()

    if USE_MOCK:
        events = mock_generate_events(user_text, today.isoformat())
        raw = json.dumps({"events": events})
        diag = {"engine":"mock","model":"mock","latency_ms":1,"input_tokens":0,"output_tokens":0}
        return events, diag, raw

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(user_text, today.isoformat())},
    ]
    t0 = time.time()
    try:
        resp = client.chat.completions.create(
            model=MODEL,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            response_format={"type": "json_object"},  # force valid JSON
            messages=messages,
        )
        raw = resp.choices[0].message.content
        events = safe_json_parse(raw)
        diag = {
            "engine": "byok",
            "model": resp.model,
            "latency_ms": int((time.time() - t0) * 1000),
            "input_tokens": getattr(resp, "usage", None).prompt_tokens if getattr(resp, "usage", None) else None,
            "output_tokens": getattr(resp, "usage", None).completion_tokens if getattr(resp, "usage", None) else None,
        }
        return events, diag, raw
    except OpenAIError as e:
        # Fallback so your app keeps working during quota/rate issues
        events = mock_generate_events(user_text, today.isoformat())
        raw = json.dumps({"events": events})
        diag = {
            "engine": "fallback-mock",
            "model": "mock",
            "latency_ms": int((time.time() - t0) * 1000),
            "error": str(e)[:200],
        }
        return events, diag, raw

In [ ]:
# 3) Quick smoke test
sample = "Dinner with John tomorrow 7pm at Boston Pizza; alex@example.com; remind me 10 minutes before."
events, diag, raw = generate_events(sample)
print("diag:", diag)
print(json.dumps(events, indent=2)[:1200])

diag: {'engine': 'mock', 'model': 'mock', 'latency_ms': 1, 'input_tokens': 0, 'output_tokens': 0}
[
  {
    "title": "Dinner with John",
    "event_date": "2025-09-19",
    "event_time": "19:00",
    "end_time": "",
    "end_date": "2025-09-19",
    "timezone": "",
    "location": "Boston Pizza",
    "invitees": [
      "alex@example.com"
    ],
    "notifications": [
      {
        "method": "popup",
        "minutes": 10
      }
    ],
    "recurrence": "",
    "confidence": 0.85
  }
]
